In [1]:
import os
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

In [2]:
band_path = []
for i in range(1, 8):
    curr_band_path = "./Landsat/LC08_L2SP_022031_20180607_20200831_02_T1/"+"LC08_L2SP_022031_20180607_20200831_02_T1_SR_B"+str(i)+".TIF"
    band_path.append(curr_band_path)



In [3]:
band_path

['./Landsat/LC08_L2SP_022031_20180607_20200831_02_T1/LC08_L2SP_022031_20180607_20200831_02_T1_SR_B1.TIF',
 './Landsat/LC08_L2SP_022031_20180607_20200831_02_T1/LC08_L2SP_022031_20180607_20200831_02_T1_SR_B2.TIF',
 './Landsat/LC08_L2SP_022031_20180607_20200831_02_T1/LC08_L2SP_022031_20180607_20200831_02_T1_SR_B3.TIF',
 './Landsat/LC08_L2SP_022031_20180607_20200831_02_T1/LC08_L2SP_022031_20180607_20200831_02_T1_SR_B4.TIF',
 './Landsat/LC08_L2SP_022031_20180607_20200831_02_T1/LC08_L2SP_022031_20180607_20200831_02_T1_SR_B5.TIF',
 './Landsat/LC08_L2SP_022031_20180607_20200831_02_T1/LC08_L2SP_022031_20180607_20200831_02_T1_SR_B6.TIF',
 './Landsat/LC08_L2SP_022031_20180607_20200831_02_T1/LC08_L2SP_022031_20180607_20200831_02_T1_SR_B7.TIF']

In [4]:
band_data = []
for i in range(0, 7):
    curr_band_path = band_path[i]
    curr_band = rxr.open_rasterio(curr_band_path, masked=True).squeeze()
    band_data.append(curr_band)

In [5]:
band_data[0]

<xarray.DataArray (y: 7871, x: 7751)>
[61008121 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 4.047e+05 4.047e+05 ... 6.372e+05 6.372e+05
  * y            (y) float64 4.741e+06 4.741e+06 ... 4.505e+06 4.504e+06
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [6]:
import pandas as pd
node = pd.read_csv("nodes.csv")

In [7]:
from pyproj import Proj, transform

inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:32616')

lat = node["lat"].tolist()
lon = node["lon"].tolist()
outx = []
outy = []
for i in range(0, len(lat)):
    x,y = transform(inProj,outProj,lon,lat)
    outx.append(x)
    outy.append(y)

/Users/carnivalbug/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/carnivalbug/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/Users/carnivalbug/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferre

In [8]:
outx = outx[0]
outy = outy[0]
node["x"]=0.0
node["y"]=0.0

In [9]:
for i in range(0, len(outx)):
    node.at[i,"x"] = outx[i]
    node.at[i,"y"] = outy[i]

In [10]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,y
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,4.636463e+06
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,4.634209e+06
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,4.641782e+06
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,4.628887e+06
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,4.635181e+06
...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06


In [11]:
node["607_1"]="-1"
node["607_2"]="-1"
node["607_3"]="-1"
node["607_4"]="-1"
node["607_5"]="-1"
node["607_6"]="-1"
node["607_7"]="-1"

In [12]:
for i in range(0, 126):
    try:
        x= node.at[i,"x"]
        y= node.at[i,"y"]
        curr_i = 0
        while(not (x>=int(band_data[0].x[curr_i]) and x<int(band_data[0].x[curr_i+1]))):
            curr_i = curr_i+1
        curr_j = 0
        while(not (y<=int(band_data[0].y[curr_j]) and y>int(band_data[0].y[curr_j+1]))):
            curr_j = curr_j+1
        node.at[i,"607_1"] = int(band_data[0][curr_j,curr_i])
        node.at[i,"607_2"] = int(band_data[1][curr_j,curr_i])
        node.at[i,"607_3"] = int(band_data[2][curr_j,curr_i])
        node.at[i,"607_4"] = int(band_data[3][curr_j,curr_i])
        node.at[i,"607_5"] = int(band_data[4][curr_j,curr_i])
        node.at[i,"607_6"] = int(band_data[5][curr_j,curr_i])
        node.at[i,"607_7"] = int(band_data[6][curr_j,curr_i])
    except:
        print(i)


In [14]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,y,607_1,607_2,607_3,607_4,607_5,607_6,607_7
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,4.636463e+06,11749,12265,14648,15054,18456,16705,14097
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,4.634209e+06,11141,11604,12557,12916,18308,15885,13509
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,4.641782e+06,8021,8874,10476,10146,18896,15080,12102
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,4.628887e+06,9613,9823,10667,10132,19217,13625,11230
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,4.635181e+06,10588,11184,12114,12573,14238,15321,14399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06,10256,10711,11490,11808,14580,14013,12507
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06,10256,10711,11490,11808,14580,14013,12507
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06,10256,10711,11490,11808,14580,14013,12507
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06,10256,10711,11490,11808,14580,14013,12507


In [14]:
## start adding 7/25

In [15]:
band_path = []
for i in range(1, 8):
    curr_band_path = "./Landsat/LC08_L2SP_022031_20180725_20200831_02_T1/"+"LC08_L2SP_022031_20180725_20200831_02_T1_SR_B"+str(i)+".TIF"
    band_path.append(curr_band_path)

In [16]:
band_path

['./Landsat/LC08_L2SP_022031_20180725_20200831_02_T1/LC08_L2SP_022031_20180725_20200831_02_T1_SR_B1.TIF',
 './Landsat/LC08_L2SP_022031_20180725_20200831_02_T1/LC08_L2SP_022031_20180725_20200831_02_T1_SR_B2.TIF',
 './Landsat/LC08_L2SP_022031_20180725_20200831_02_T1/LC08_L2SP_022031_20180725_20200831_02_T1_SR_B3.TIF',
 './Landsat/LC08_L2SP_022031_20180725_20200831_02_T1/LC08_L2SP_022031_20180725_20200831_02_T1_SR_B4.TIF',
 './Landsat/LC08_L2SP_022031_20180725_20200831_02_T1/LC08_L2SP_022031_20180725_20200831_02_T1_SR_B5.TIF',
 './Landsat/LC08_L2SP_022031_20180725_20200831_02_T1/LC08_L2SP_022031_20180725_20200831_02_T1_SR_B6.TIF',
 './Landsat/LC08_L2SP_022031_20180725_20200831_02_T1/LC08_L2SP_022031_20180725_20200831_02_T1_SR_B7.TIF']

In [17]:
band_data = []
for i in range(0, 7):
    curr_band_path = band_path[i]
    curr_band = rxr.open_rasterio(curr_band_path, masked=True).squeeze()
    band_data.append(curr_band)

In [18]:
band_data[0]

<xarray.DataArray (y: 7871, x: 7741)>
[60929411 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 4.05e+05 4.05e+05 4.051e+05 ... 6.372e+05 6.372e+05
  * y            (y) float64 4.741e+06 4.741e+06 ... 4.505e+06 4.504e+06
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [19]:
node["725_1"]="-1"
node["725_2"]="-1"
node["725_3"]="-1"
node["725_4"]="-1"
node["725_5"]="-1"
node["725_6"]="-1"
node["725_7"]="-1"

In [20]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,...,607_5,607_6,607_7,725_1,725_2,725_3,725_4,725_5,725_6,725_7
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,...,18456,16705,14097,-1,-1,-1,-1,-1,-1,-1
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,...,18308,15885,13509,-1,-1,-1,-1,-1,-1,-1
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,...,18896,15080,12102,-1,-1,-1,-1,-1,-1,-1
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,...,19217,13625,11230,-1,-1,-1,-1,-1,-1,-1
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,...,14238,15321,14399,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14580,14013,12507,-1,-1,-1,-1,-1,-1,-1
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14580,14013,12507,-1,-1,-1,-1,-1,-1,-1
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14580,14013,12507,-1,-1,-1,-1,-1,-1,-1
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14580,14013,12507,-1,-1,-1,-1,-1,-1,-1


In [21]:
for i in range(0, 126):
    try:
        x= node.at[i,"x"]
        y= node.at[i,"y"]
        curr_i = 0
        while(not (x>=int(band_data[0].x[curr_i]) and x<int(band_data[0].x[curr_i+1]))):
            curr_i = curr_i+1
        curr_j = 0
        while(not (y<=int(band_data[0].y[curr_j]) and y>int(band_data[0].y[curr_j+1]))):
            curr_j = curr_j+1
        node.at[i,"725_1"] = int(band_data[0][curr_j,curr_i])
        node.at[i,"725_2"] = int(band_data[1][curr_j,curr_i])
        node.at[i,"725_3"] = int(band_data[2][curr_j,curr_i])
        node.at[i,"725_4"] = int(band_data[3][curr_j,curr_i])
        node.at[i,"725_5"] = int(band_data[4][curr_j,curr_i])
        node.at[i,"725_6"] = int(band_data[5][curr_j,curr_i])
        node.at[i,"725_7"] = int(band_data[6][curr_j,curr_i])
    except:
        print(i)

In [22]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,...,607_5,607_6,607_7,725_1,725_2,725_3,725_4,725_5,725_6,725_7
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,...,18456,16705,14097,10794,11674,12999,13479,16245,16717,13808
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,...,18308,15885,13509,10653,10942,12222,12213,18383,16121,13780
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,...,18896,15080,12102,8521,9041,10183,10074,17818,15399,12232
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,...,19217,13625,11230,9351,9670,10336,9919,16974,13343,10924
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,...,14238,15321,14399,10178,10764,11464,11915,13928,14605,13840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14580,14013,12507,10173,10591,11488,11780,14626,13502,12356
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14580,14013,12507,10173,10591,11488,11780,14626,13502,12356
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14580,14013,12507,10173,10591,11488,11780,14626,13502,12356
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14580,14013,12507,10173,10591,11488,11780,14626,13502,12356


In [26]:
## start adding 9/11

In [23]:
band_path = []
for i in range(1, 8):
    curr_band_path = "./Landsat/LC08_L2SP_022031_20180911_20200830_02_T1/"+"LC08_L2SP_022031_20180911_20200830_02_T1_SR_B"+str(i)+".TIF"
    band_path.append(curr_band_path)

In [24]:
band_path

['./Landsat/LC08_L2SP_022031_20180911_20200830_02_T1/LC08_L2SP_022031_20180911_20200830_02_T1_SR_B1.TIF',
 './Landsat/LC08_L2SP_022031_20180911_20200830_02_T1/LC08_L2SP_022031_20180911_20200830_02_T1_SR_B2.TIF',
 './Landsat/LC08_L2SP_022031_20180911_20200830_02_T1/LC08_L2SP_022031_20180911_20200830_02_T1_SR_B3.TIF',
 './Landsat/LC08_L2SP_022031_20180911_20200830_02_T1/LC08_L2SP_022031_20180911_20200830_02_T1_SR_B4.TIF',
 './Landsat/LC08_L2SP_022031_20180911_20200830_02_T1/LC08_L2SP_022031_20180911_20200830_02_T1_SR_B5.TIF',
 './Landsat/LC08_L2SP_022031_20180911_20200830_02_T1/LC08_L2SP_022031_20180911_20200830_02_T1_SR_B6.TIF',
 './Landsat/LC08_L2SP_022031_20180911_20200830_02_T1/LC08_L2SP_022031_20180911_20200830_02_T1_SR_B7.TIF']

In [25]:
band_data = []
for i in range(0, 7):
    curr_band_path = band_path[i]
    curr_band = rxr.open_rasterio(curr_band_path, masked=True).squeeze()
    band_data.append(curr_band)

In [26]:
band_data[0]

<xarray.DataArray (y: 7871, x: 7751)>
[61008121 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 4.041e+05 4.041e+05 ... 6.366e+05 6.366e+05
  * y            (y) float64 4.741e+06 4.741e+06 ... 4.505e+06 4.504e+06
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [27]:
node["911_1"]="-1"
node["911_2"]="-1"
node["911_3"]="-1"
node["911_4"]="-1"
node["911_5"]="-1"
node["911_6"]="-1"
node["911_7"]="-1"

In [31]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,...,725_5,725_6,725_7,911_1,911_2,911_3,911_4,911_5,911_6,911_7
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,...,16245,16717,13808,11671,12782,13847,14832,18264,16076,13251
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,...,18383,16121,13780,10507,10915,11742,11695,18905,15359,12819
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,...,17818,15399,12232,7844,8273,9789,9509,18612,14102,11404
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,...,16974,13343,10924,9367,9297,9982,9609,16513,13009,10662
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,...,13928,14605,13840,10362,10738,11772,12227,13295,14416,13675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14626,13502,12356,9694,9963,10837,10937,13393,13428,12186
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14626,13502,12356,9694,9963,10837,10937,13393,13428,12186
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14626,13502,12356,9694,9963,10837,10937,13393,13428,12186
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14626,13502,12356,9694,9963,10837,10937,13393,13428,12186


In [32]:
node.to_csv("node_with_band.csv")